<a href="https://colab.research.google.com/github/yudintsev-sergey/Neyro/blob/main/Hokku_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RNN-сеть для генерации "заготовок" для хайку**

Отключаем GPU

In [59]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Импортируем библиотеки

In [60]:
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

Подключаем диск

In [61]:
from google.colab import drive
drive.mount("/ShardDrives/")

os.chdir('/ShardDrives/MyDrive/Colab Notebooks/Hokku/')

Drive already mounted at /ShardDrives/; to attempt to forcibly remount, call drive.mount("/ShardDrives/", force_remount=True).


Читаем текст

In [62]:
with open('hokku.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

Разбиваем на слова

In [63]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

Проверяем просто что получилось (печатаем для примера начало списка). Цифры - частотность слова.

In [64]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('на', 27), ('голой', 1), ('ветке', 2), ('ворон', 1), ('сидит', 1), ('одиноко', 2), ('осенний', 3), ('вечер', 3), ('в', 31), ('небе', 1)]


Конвертируем текст в последовательность индексов слов, которые входят в текст

In [65]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

Строим модель и компилируем ее

In [66]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_1 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434280 (1.66 MB)
Trainable params: 434280 (1.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Обучаем

In [67]:
history = model.fit(X, Y, batch_size=32, epochs=50, verbose=0)

Функция построения (прогнозирования) фразы

In [68]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


Найдем три случайные слова по случайным индексам в пределах 599 (длина словарной базы).
Потом их можно скопировать и вставить в функцию buildPhrase().

In [69]:
m = np.random.randint(599, size=3)
print(dist[m[0]][0], dist[m[1]][0], dist[m[2]][0])

битвы брови остаешься


Voila!

In [70]:
res = buildPhrase("битвы брови остаешься")
print(res)

1/1 [==============================] - 0s 22ms/step
битвы брови остаешься цветок грабитель две завял предостерег для летом снега тушью рядом вокруг сливы мой под людей глухой… свежий палубе деревню разлилась


Из полученной абракадабры можно слепить хайку... Удачи!!! :0))